# Class 2 - Part 1: Deploy CNN as API

## Objective
In this session, you will:
1. Wrap the trained CNN model as a REST API using FastAPI
2. Create a `/predict` endpoint that accepts images and returns predictions
3. Implement input validation and error handling
4. Test the API using Python requests
5. Handle edge cases (wrong shape, corrupt images)

## Key Concepts
- **API Design**: How ML models transition from notebooks to production
- **Input Validation**: Ensuring only valid data reaches the model
- **Error Handling**: Graceful failure and meaningful error messages
- **Model Versioning**: Tracking which model version is serving predictions

## Why APIs?

Models in production aren't run in notebooks. They need to be:
- **Accessible**: Other applications need to use them
- **Scalable**: Handle many concurrent requests
- **Reliable**: Never crash the entire system
- **Monitored**: Track performance and usage
- **Versioned**: Support multiple model versions

**REST API** is the standard way to expose ML models to the world.

## Step 1: Setup & Load Trained Model

In [2]:
import os
import io
import json
import base64
import numpy as np
from pathlib import Path
from PIL import Image

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet18

# FastAPI and testing
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.responses import JSONResponse
import uvicorn
import requests
from pydantic import BaseModel

print("✅ Libraries imported")

✅ Libraries imported


In [3]:
# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# Class names from training
class_names = ['animal', 'name_board', 'other_vehicle', 'pedestrian', 'pothole', 'road_sign', 'speed_breaker']
num_classes = len(class_names)

print(f"✅ Classes: {class_names}")

Device: cpu
✅ Classes: ['animal', 'name_board', 'other_vehicle', 'pedestrian', 'pothole', 'road_sign', 'speed_breaker']


In [4]:
# Create a simple CNN model (same as in Class 1)
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.resnet = resnet18(pretrained=False)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)
    
    def forward(self, x):
        return self.resnet(x)

# Create and move to device
model = CNNModel(num_classes).to(device)
model.eval()  # Set to evaluation mode

print("✅ Model created (using random weights for demo)")
print("   Note: In production, you would load pre-trained weights")

c:\Users\Lucifer\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lucifer\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


✅ Model created (using random weights for demo)
   Note: In production, you would load pre-trained weights


In [5]:
# Data transformation (same as training)
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

print("✅ Image transformation pipeline ready")

✅ Image transformation pipeline ready


## Step 2: Create FastAPI Application

In [6]:
# Define API response model using Pydantic
class PredictionResponse(BaseModel):
    """Response schema for model predictions"""
    prediction: str  # Class name
    confidence: float  # Probability
    class_probabilities: dict  # All class probabilities
    model_version: str
    latency_ms: float

# Create FastAPI app
app = FastAPI(
    title="ADAS Model API",
    description="CNN model for detecting ADAS objects",
    version="1.0.0"
)

print("✅ FastAPI application created")

✅ FastAPI application created


In [7]:
import time

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    """
    Predict class of uploaded image.
    
    Args:
        file: Image file (JPEG, PNG)
    
    Returns:
        PredictionResponse with prediction, confidence, and model version
    """
    start_time = time.time()
    
    try:
        # Step 1: Validate file type
        allowed_extensions = {'.jpg', '.jpeg', '.png', '.gif'}
        file_ext = Path(file.filename).suffix.lower()
        
        if file_ext not in allowed_extensions:
            raise HTTPException(
                status_code=400,
                detail=f"Invalid file type: {file_ext}. Allowed: {allowed_extensions}"
            )
        
        # Step 2: Read and validate image
        contents = await file.read()
        
        try:
            image = Image.open(io.BytesIO(contents)).convert('RGB')
        except Exception as e:
            raise HTTPException(
                status_code=400,
                detail=f"Could not open image: {str(e)}"
            )
        
        # Step 3: Validate image dimensions
        if image.size[0] < 32 or image.size[1] < 32:
            raise HTTPException(
                status_code=400,
                detail=f"Image too small: {image.size}. Minimum: 32x32"
            )
        
        # Step 4: Transform and batch
        image_tensor = transform(image).unsqueeze(0).to(device)  # Add batch dimension
        
        # Step 5: Model inference
        with torch.no_grad():
            outputs = model(image_tensor)
            probabilities = torch.softmax(outputs, dim=1)
            confidence, predicted_idx = torch.max(probabilities, 1)
        
        # Step 6: Format response
        predicted_class = class_names[predicted_idx.item()]
        confidence_value = confidence.item()
        
        # Get all class probabilities
        class_probs = {}
        for idx, class_name in enumerate(class_names):
            class_probs[class_name] = float(probabilities[0, idx].item())
        
        # Calculate latency
        latency = (time.time() - start_time) * 1000  # Convert to ms
        
        return PredictionResponse(
            prediction=predicted_class,
            confidence=round(confidence_value, 4),
            class_probabilities=class_probs,
            model_version="v1.0",
            latency_ms=round(latency, 2)
        )
    
    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Internal server error: {str(e)}"
        )

print("✅ /predict endpoint defined")

✅ /predict endpoint defined


In [8]:
# Health check endpoint
@app.get("/health")
async def health_check():
    """
    Check if API is alive and model is loaded.
    """
    return {
        "status": "healthy",
        "model_version": "v1.0",
        "classes": class_names,
        "device": str(device)
    }

# Model info endpoint
@app.get("/info")
async def model_info():
    """
    Get model information.
    """
    total_params = sum(p.numel() for p in model.parameters())
    
    return {
        "model_name": "ResNet-18 ADAS Detector",
        "version": "1.0",
        "num_classes": num_classes,
        "classes": class_names,
        "input_shape": "128x128x3",
        "total_parameters": total_params,
        "device": str(device)
    }

print("✅ Health check and info endpoints defined")

✅ Health check and info endpoints defined


## Step 3: Run API Server

⚠️ **IMPORTANT**: The code below starts a server. You might want to run it in a separate terminal instead of in this notebook.

In [9]:
# Start server (runs indefinitely until stopped)
# Note: In Jupyter, you typically run this in a separate terminal:
# uvicorn <script_name>:app --reload --port 8000

# For notebook demo, we'll create a test client instead
from fastapi.testclient import TestClient

client = TestClient(app)
print("✅ Test client created (FastAPI TestClient)")

✅ Test client created (FastAPI TestClient)


## Step 4: Test API Endpoints

In [10]:
# Test 1: Health Check
print("\n" + "="*70)
print("TEST 1: Health Check Endpoint")
print("="*70)

response = client.get("/health")
print(f"Status Code: {response.status_code}")
print(f"Response: {json.dumps(response.json(), indent=2)}")


TEST 1: Health Check Endpoint
Status Code: 200
Response: {
  "status": "healthy",
  "model_version": "v1.0",
  "classes": [
    "animal",
    "name_board",
    "other_vehicle",
    "pedestrian",
    "pothole",
    "road_sign",
    "speed_breaker"
  ],
  "device": "cpu"
}


In [ ]:
# Test 2: Model Info
print("\n" + "="*70)
print("TEST 2: Model Info Endpoint")
print("="*70)

response = client.get("/info")
print(f"Status Code: {response.status_code}")
info = response.json()
for key, value in info.items():
    if key != 'classes':
        print(f"{key}: {value}")
    else:
        print(f"{key}: {', '.join(value)}")

In [ ]:
# Test 3: Prediction with Valid Image
print("\n" + "="*70)
print("TEST 3: Prediction with Synthetic Image")
print("="*70)

# Create a dummy image
dummy_image = Image.new('RGB', (100, 100), color='red')
img_bytes = io.BytesIO()
dummy_image.save(img_bytes, format='PNG')
img_bytes.seek(0)

response = client.post(
    "/predict",
    files={"file": ("test_image.png", img_bytes, "image/png")}
)

print(f"Status Code: {response.status_code}")
if response.status_code == 200:
    result = response.json()
    print(f"Prediction: {result['prediction']}")
    print(f"Confidence: {result['confidence']:.4f}")
    print(f"Latency: {result['latency_ms']:.2f} ms")
    print(f"Model Version: {result['model_version']}")
else:
    print(f"Error: {response.text}")

In [ ]:
# Test 4: Invalid File Type (should fail)
print("\n" + "="*70)
print("TEST 4: Invalid File Type (Edge Case)")
print("="*70)

# Try to upload a text file
response = client.post(
    "/predict",
    files={"file": ("test.txt", io.BytesIO(b"not an image"), "text/plain")}
)

print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")
print("✅ API correctly rejected invalid file type")

In [ ]:
# Test 5: Corrupt/Invalid Image (should fail)
print("\n" + "="*70)
print("TEST 5: Corrupt Image (Edge Case)")
print("="*70)

# Send corrupted image data
response = client.post(
    "/predict",
    files={"file": ("corrupt.png", io.BytesIO(b"\x89PNG invalid data"), "image/png")}
)

print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")
print("✅ API correctly rejected corrupted image")

In [ ]:
# Test 6: Image Too Small (should fail)
print("\n" + "="*70)
print("TEST 6: Image Too Small (Edge Case)")
print("="*70)

# Create a very small image
tiny_image = Image.new('RGB', (16, 16), color='blue')
img_bytes = io.BytesIO()
tiny_image.save(img_bytes, format='PNG')
img_bytes.seek(0)

response = client.post(
    "/predict",
    files={"file": ("tiny.png", img_bytes, "image/png")}
)

print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")
print("✅ API correctly rejected too-small image")

## Step 5: API Testing Summary

In [ ]:
print("\n" + "="*70)
print("🎯 API TESTING SUMMARY")
print("="*70)

test_results = {
    "✅ Health Check": "API is responsive",
    "✅ Model Info": "Metadata correctly returned",
    "✅ Valid Image": "Prediction successful with latency",
    "✅ File Type Validation": "Rejected invalid file formats",
    "✅ Image Corruption Check": "Rejected corrupt images",
    "✅ Size Validation": "Enforced minimum image dimensions"
}

for test, result in test_results.items():
    print(f"{test}: {result}")

print("\n" + "="*70)
print("🚀 KEY TAKEAWAYS:")
print("="*70)
print("""
1. APIs transform models into enterprise-ready services
2. INPUT VALIDATION is crucial - never trust user input
3. ERROR HANDLING should be graceful with meaningful messages
4. Endpoints like /health and /info are standard practice
5. Latency tracking helps identify performance issues
6. Model versioning allows A/B testing and rollbacks
""")

## Running the API Locally

To run this API with FastAPI, create a file named `api.py` with the model and endpoints, then run:

```bash
pip install fastapi uvicorn python-multipart
uvicorn api:app --reload --port 8000
```

Then test with:
```python
import requests
files = {'file': open('image.jpg', 'rb')}
response = requests.post('http://localhost:8000/predict', files=files)
print(response.json())
```

Or use **Postman**: POST to `http://localhost:8000/predict` with form-data file upload.